<a id="toc"></a>

# <u>Table of Contents</u>

1.) [Setup](#setup)  
&nbsp;&nbsp;&nbsp;&nbsp; 1.1.) [Imports](#imports)   
&nbsp;&nbsp;&nbsp;&nbsp; 1.2.) [Helpers](#helpers)   
&nbsp;&nbsp;&nbsp;&nbsp; 1.3.) [Load data](#load)   
2.) [Datetime](#datetime)  
3.) [Speakers](#speakers)  
4.) [Transcript](#transcript)  
5.) [Save to CSV](#save)  

---
<a id="setup"></a>

# [^](#toc) <u>Setup</u>

<a id="imports"></a>

### [^](#toc) Imports

In [1]:
### Standard imports
import pandas as pd
import numpy as np

### Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

import datetime

# Helps convert String representation of list into a list
import ast

### Removes warnings that occassionally show in imports
import warnings
warnings.filterwarnings('ignore')

<a id="helpers"></a>

### [^](#toc) Helpers

In [2]:
def string_literal(x):
    if x is np.nan:
        return np.nan
    else:
        try:
            return ast.literal_eval(x)
        except:
            return x

<a id="load"></a>

### [^](#toc) Load data

In [15]:
df = pd.read_csv("data/PBS_full_unedited.csv")
for col in ["Transcript", "Story", "Speakers"]:
    df[col] = df[col].map(string_literal)
    
df.head()

,URL,Story,Date,Title,Transcript,Speakers,Number of Comments
0,https://www.pbs.org/newshour/show/news-wrap-tr...,"In our news wrap Monday, President Trump's sea...","Jul 2, 2018 6:50 PM EDT",News Wrap: Trump interviews Supreme Court cand...,"[[Judy Woodruff, [ President Trump’s search fo...","{Justin Herdman, President Donald Trump, Man (...",0.0
1,https://www.pbs.org/newshour/show/elected-in-a...,Mexican president-elect Andrés Manuel López Ob...,"Jul 2, 2018 6:45 PM EDT","Elected by a landslide, can Mexico’s López Obr...","[[Judy Woodruff, [ After two previous runs for...","{Alfonso Romo, Diana Mercado (through translat...",0.0
2,https://www.pbs.org/newshour/show/will-u-s-mex...,There are enormous expectations facing the new...,"Jul 2, 2018 6:43 PM EDT",Will U.S.-Mexico policy tensions change under ...,"[[Judy Woodruff, [ And now perspective from fo...","{Roberta Jacobson, Judy Woodruff}",0.0
3,https://www.pbs.org/newshour/show/yemens-spira...,One of the poorest countries in the Middle Eas...,"Jul 2, 2018 6:40 PM EDT",Yemen’s spiraling hunger crisis is a man-made ...,"[[Judy Woodruff, [ The “NewsHour” has reported...","{Naimi (through translator), Lise Grande, Yahy...",0.0
4,https://www.pbs.org/newshour/show/livingwhileb...,A profusion of national incidents in which whi...,"Jul 2, 2018 6:35 PM EDT",#LivingWhileBlack: How does racial bias lead t...,"[[Judy Woodruff, [ A number of recent incident...","{Yamiche Alcindor, Woman, Judy Woodruff, Derri...",0.0


---
<a id="datetime"></a>

# [^](#toc) <u>Datetime</u>

First let's clear off whitespace and \n characters

In [16]:
df["Date"] = df["Date"].map(lambda x: x.strip())

#### Time Zone

It appears, the time is always posted in EDT

In [17]:
df["temp"] = df.Date.map(lambda x: x[-3:])
vc = df["temp"].value_counts()

df.drop("temp", axis=1, inplace=True)
vc

EDT    17617
Name: temp, dtype: int64

### Updated times

There are only 4 videos that were updated.  I don't think this is very interesting so I'm just going to ignore all updated times.

I understand why someone would like this data, but I don't need it.

In [18]:
df.temp = df.Date.map(lambda x: int("Updated" in x))
num_updated = len(df[df.temp == 1])
print(f"{num_updated} clips have had information updated")

df.Date = df.Date.map(lambda x: x.split("\n")[0])

4 clips have had information updated


### Convert to Datetime

See this Stack Exchange [link](https://english.stackexchange.com/questions/35315/what-is-the-proper-name-for-am-and-pm?newreg=2d443a2ca9dc4ba6abbe1a1e01e4af4b) haha.  I was honored by the line...

    software developers think about naming variables properly. It is built into the Object-Oriented mindset. Jader, a commenter, said it well: "It's funny that the question intrinsically is not programming related, but all programmers can understand why you posted it here."
    
I just considered myself a physicist!

In [19]:
def format_datetime(x):
    return datetime.datetime.strptime(x[:-4], '%b %d, %Y %H:%M %p')

df.Date = df.Date.map(format_datetime)

### And we're done with times!

---
<a id="speakers"></a>

# [^](#toc) <u>Speakers</u>

In [15]:
for row in df.loc[df.Speakers.isnull(), 'Speakers'].index:
    df.at[row, 'Speakers'] = {"Unknown"}

In [18]:
sum(df.Speakers.map(lambda x: 1 if "Judy Woodruff" in x else 0))

1031

In [17]:
len(set.union(*df.Speakers))

32726

---
<a id="transcript"></a>

# [^](#toc) <u>Transcript</u>

In [20]:
for row in df.loc[df.Transcript.isnull(), 'Transcript'].index:
    df.at[row, 'Transcript'] = ["Unknown"]

In [22]:
df.Transcript

0        [[Judy Woodruff, [ President Trump’s search fo...
1        [[Judy Woodruff, [ After two previous runs for...
2        [[Judy Woodruff, [ And now perspective from fo...
3        [[Judy Woodruff, [ The “NewsHour” has reported...
4        [[Judy Woodruff, [ A number of recent incident...
5        [[Judy Woodruff, [ And now back to the looming...
6        [[Judy Woodruff, [ On our Bookshelf tonight, w...
7        [[HARI SREENIVASEN, [ Trade is one of the area...
8        [[JEFF GREENFIELD, [ It has the look and feel ...
9        [[CAMERON ESPOSITO, [ “And I don’t totally rem...
10       [[LISA DESJARDINS, [ Despite court orders it i...
11       [[IVETTE FELICIANO, [ Success did not come eas...
12       [[Judy Woodruff, [ President Trump now says he...
13       [[Judy Woodruff, [ This week, a federal judge ...
14       [[Judy Woodruff, [ How will the Trump administ...
15       [[Judy Woodruff, [ And now, the aftermath of t...
16       [[Judy Woodruff, [ While the U.S. grapples wit.

---
<a id="save"></a>

# [^](#toc) <u>Save to CSV</u>

Now that we've cleaned up our data a bit, let's save it into a different CSV file.

In [ ]:
df.to_csv("data/PBS-newhour-clean.csv")